### This project explores content-based filtering for recommending movies to users in Netflix. Used tdif vectorizer on the description.
Next, will explore Credits and Genres Based Recommender by using the credits and genres data. Check out Metadata based recommender from this to apply this system. https://www.kaggle.com/code/rounakbanik/movie-recommender-systems

## Content Based filtering

Content Based filtering 
guide https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system
dataset 

Content-Based Filtering is a type of recommender system that suggests items to users based on their preferences and past behavior. It works by analyzing the characteristics or features of the items and comparing them to the user's profile or history to make recommendations.

For example, in a movie recommendation system, the system may analyze the genre, actors, director, and other attributes of the movies a user has liked and recommend similar movies with similar attributes.
dataset source https://www.kaggle.com/datasets/victorsoeiro/netflix-tv-shows-and-movies


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
df_titles=pd.read_csv('C:/Users/Shai/OneDrive/Documents/Data science/Netflix dataset/titles.csv')
df_credits=pd.read_csv('C:/Users/Shai/OneDrive/Documents/Data science/Netflix dataset/credits.csv')
print(df_titles.shape)
print(df_credits.shape)
df_titles.head()

(5850, 15)
(77801, 6)


,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


TF-IDF (Term Frequency-Inverse Document Frequency) is a numerical statistic used to measure the importance of a word in a document collection. It combines measures of how often a term appears in a document (TF) and how common or rare it is across all documents (IDF) to give a higher weight to important terms. TF-IDF is widely used in natural language processing and machine learning for tasks such as text classification and ranking.

In [3]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df_titles['description'] = df_titles['description'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df_titles['description'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(5850, 21061)

Compute a similarity score. There are several candidates for this; such as the euclidean, the Pearson and the cosine similarity scores. The later was used on this project will experiment with other scores next time.

In [4]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [5]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(df_titles.index, index=df_titles['title']).drop_duplicates()

In [6]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    # enumerate - returns an iterator of tuples (cosine similarity score and index) each tuple contains a count (starting from zero by default) and an element from the iterable(similarity score) that is being enumerated.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df_titles[['title', 'description']].iloc[movie_indices]

In [7]:
get_recommendations('Inception')

,title,description
5569,Homunculus,A homeless man meets a medical school student ...
4935,Glamour Girls,Their life as luxury escorts offered these wom...
2246,Familiar Wife,After receiving a bizarre chance to go back in...
3171,The Wrong Missy,A guy meets the woman of his dreams and invite...
4463,Dollar,Tarek is given the task to come up with a mill...
5043,Stowaway,A three-person crew on a mission to Mars faces...
4222,Into the Beat,A ballerina discovers hip-hop by chance and is...
4699,Spiderhead,"In the near future, convicts are offered the c..."
4089,Bard of Blood,"Political espionage thriller based on ""Bard of..."
4281,Once Again,After a chance phone call leads to daily conve...


## Credits and Genres Based Recommender

A credits and genres-based recommender system utilizes information about credits (e.g., directors, actors, writers) and genres of movies or other media to suggest similar items to users. 

In [10]:
df_credits.head()

,credits_id,person_id,id,name,character,role
0,1,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,2,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,3,7064,tm84618,Albert Brooks,Tom,ACTOR
3,4,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,5,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


In [ ]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [8]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['name', 'director', 'genres']
for feature in features:
    df_merged[feature] = df_merged[feature].apply(literal_eval)

NameError: name 'df_merged' is not defined

Name- will be pivoted first

In [ ]:
# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
    for i in x:
        if i['role'] == 'Director':
            return i['name']
    return np.nan

In [ ]:
# Define new director, cast, genres and keywords features that are in a suitable form.
df_credit['director'] = df_credit['name'].apply(get_director)

In [ ]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [ ]:
features = ['actor']
for feature in features:
    df_credits[feature] = df_credits[feature].apply(get_list)